# Poblar BD de Neo4j

In [7]:
from neo4j import GraphDatabase
import csv

# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "postgres"

# CSV file path
csv_file = "nodes.csv"

# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
CREATE (n:Nodos)
SET n.type = row.type,
    n.country = row.country,
    n.id = row.id
"""

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))

# Close the Neo4j driver
driver.close()

In [8]:
from neo4j import GraphDatabase
import csv

# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "postgres"

# CSV file path
csv_file = "links.csv"

# Cypher query to create relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source:Nodos), (target:Nodos)
WHERE source.id = row.source AND target.id = row.target AND row.type = 'ownership'
CREATE (source)-[:ownership]->(target)
"""

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))

# Cypher query to create relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source:Nodos), (target:Nodos)
WHERE source.id = row.source AND target.id = row.target AND row.type = 'membership'
CREATE (source)-[:membership]->(target)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))
    
# Cypher query to create relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source:Nodos), (target:Nodos)
WHERE source.id = row.source AND target.id = row.target AND row.type = 'family_relationship'
CREATE (source)-[:family_relationship]->(target)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))


# Cypher query to create relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source:Nodos), (target:Nodos)
WHERE source.id = row.source AND target.id = row.target AND row.type = 'partnership'
CREATE (source)-[:partnership]->(target)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))
# Close the Neo4j driver
driver.close()


# Gráfico Nodos Sospechosos con sus Conexiones de Nivel 1

In [26]:
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node:Nodos)-[r:ownership|membership|partnership|family_relationship]-(n:Nodos)
WHERE node.id = "Mar de la Vida OJSC" OR
	    node.id = "979893388" OR
        node.id = "Oceanfront Oasis Inc Carriers" OR
        node.id = "8327"
RETURN node, r, n
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], record["n"]["id"], record["r"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes and edges to the network
for source, target, relation_type in graph_data:
    if source == "Mar de la Vida OJSC":
        network.add_node(source,title = source ,color = "red")
    elif source == "979893388":
        network.add_node(source,title = source ,color = "green")
    elif source == "Oceanfront Oasis Inc Carriers":
        network.add_node(source,title = source ,color = "orange")
    elif source == "8327":
        network.add_node(source,title = source ,color = "purple")    
    else:
        network.add_node(source, title = source)
    if target == "Mar de la Vida OJSC":
        network.add_node(target,title = target ,color = "red")
    elif target == "979893388":
        network.add_node(target,title = target ,color = "green")
    elif target == "Oceanfront Oasis Inc Carriers":
        network.add_node(target,title = target ,color = "orange")
    elif target == "8327":
        network.add_node(target,title = target ,color = "purple")    
    else:
        network.add_node(target,title = target)
    network.add_edge(source, target, title=relation_type)

# Visualize the network
network.show("graph.html")

# Close the Neo4j driver connection
driver.close()

graph.html


# Gráfico Conexiones de Nivel 2

In [30]:
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (n)-[r]-(t)
WHERE SIZE((n)--()) >= 2
RETURN n, r, t
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["n"]["id"], record["t"]["id"], record["r"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes and edges to the network
for source, target, relation_type in graph_data:
    if source in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"] or target in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"]:
        if source == "Mar de la Vida OJSC":
            network.add_node(source,title = source ,color = "red")
        elif source == "979893388":
            network.add_node(source,title = source ,color = "green")
        elif source == "Oceanfront Oasis Inc Carriers":
            network.add_node(source,title = source ,color = "orange")
        elif source == "8327":
            network.add_node(source,title = source ,color = "purple")    
        else:
            network.add_node(source, title = source)
        if target == "Mar de la Vida OJSC":
            network.add_node(target,title = target ,color = "red")
        elif target == "979893388":
            network.add_node(target,title = target ,color = "green")
        elif target == "Oceanfront Oasis Inc Carriers":
            network.add_node(target,title = target ,color = "orange")
        elif target == "8327":
            network.add_node(target,title = target ,color = "purple")    
        else:
            network.add_node(target,title = target)
        network.add_edge(source, target, title=relation_type)

# Visualize the network
network.show("graph_lvl2.html")

# Close the Neo4j driver connection
driver.close()

graph_lvl2.html
